# Stock Prediction Analysis Notebook

This notebook walks through data loading, visualization, preprocessing, model training, and evaluation for stock price prediction.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from data_loader import load_stock_data, visualize_stock_data, prepare_data
from models import LinearRegressionModel, RandomForestModel, LSTMModel

In [ ]:
# Load sample stock data
ticker = 'AAPL'
data = load_stock_data(ticker)
data.head()

In [ ]:
# Plot close price time series
visualize_stock_data(data, ticker, output_dir='results_notebook')
plt.show()

In [ ]:
# Prepare data for modeling
X_train, X_test, y_train, y_test, scaler = prepare_data(data, sequence_length=60, test_size=0.2)
X_train.shape, X_test.shape

In [ ]:
# Train a quick Linear Regression and evaluate
lr = LinearRegressionModel()
lr.fit(X_train, y_train)
metrics, y_pred = lr.evaluate(X_test, y_test, scaler)
metrics

In [ ]:
# Train and evaluate Random Forest model
rf = RandomForestModel(n_estimators=100, max_depth=20)
rf.fit(X_train, y_train)
rf_metrics, rf_pred = rf.evaluate(X_test, y_test, scaler)
rf_metrics

In [ ]:
# Train and evaluate LSTM model
lstm = LSTMModel((X_train.shape[1], 1))
lstm.fit(X_train, y_train, epochs=20, batch_size=32)
lstm_metrics, lstm_pred = lstm.evaluate(X_test, y_test, scaler)
lstm_metrics

In [ ]:
# Plot LSTM training loss
from models import plot_training_loss
plot_training_loss(lstm.history)
plt.show()

In [ ]:
# Compare all models' performance
from models import compare_models
comparison_df = compare_models([lr, rf, lstm], X_test, y_test, scaler)
comparison_df

In [ ]:
# Ablation study on sequence length
import pandas as pd
sequence_lengths = [10, 30, 60, 90, 120]
results = []
for seq in sequence_lengths:
    Xtr, Xte, ytr, yte, _ = prepare_data(data, sequence_length=seq, test_size=0.2)
    model = LinearRegressionModel()
    model.fit(Xtr, ytr)
    m, _ = model.evaluate(Xte, yte, scaler)
    results.append({'Sequence Length': seq, 'RMSE_real': m['RMSE_real']})
ablation_df = pd.DataFrame(results)
ablation_df

In [ ]:
# Future day prediction
last_seq = data['Close'].values[-60:].reshape(1, 60, 1)
for model in [lr, rf, lstm]:
    pred = model.predict(last_seq)
    price = scaler.inverse_transform(pred.reshape(-1,1))[0,0]
    print(f'{model.name}: ${price:.2f}')